Import

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE

Data load

In [3]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

Data Preprocessing

In [4]:
X = train.drop(columns=['ID', 'Cancer'])
y = train['Cancer']

x_test = test.drop('ID', axis=1)

categorical_features = [col for col in X.columns if X[col].dtype == 'object']
for col in categorical_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    for val in np.unique(x_test[col]):
        if val not in le.classes_:
            le.classes_ = np.append(le.classes_, val)
    x_test[col] = le.transform(x_test[col])

Train

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

def train_and_eval(X_tr, y_tr, X_val, y_val, label):
    model = XGBClassifier(random_state=42)
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred)
    print(f"[{label}] Validation F1-score: {f1:.4f}")
    return model, f1

# (1) SMOTE 미적용
model_raw, f1_raw = train_and_eval(X_train, y_train, X_val, y_val, "RAW")

# (2) SMOTE 적용
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
model_smote, f1_smote = train_and_eval(X_train_smote, y_train_smote, X_val, y_val, "SMOTE")

# SMOTE 적용 여부에 따라 최종 학습 데이터 구성
if f1_smote >= f1_raw:
    smote_full = SMOTE(random_state=42)
    X_final, y_final = smote_full.fit_resample(X, y)
else:
    X_final, y_final = X, y



# 최종 모델 학습
final_model = XGBClassifier(random_state=42)
final_model.fit(X_final, y_final)

[RAW] Validation F1-score: 0.2953
[SMOTE] Validation F1-score: 0.3311
F1-score on training data with interactions: 0.8396
F1-score on validation data with interactions: 0.3247


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

Predict

In [6]:
final_pred = final_model.predict(x_test)

ValueError: feature_names mismatch: ['Age', 'Gender', 'Country', 'Race', 'Family_Background', 'Radiation_History', 'Iodine_Deficiency', 'Smoke', 'Weight_Risk', 'Diabetes', 'Nodule_Size', 'TSH_Result', 'T4_Result', 'T3_Result'] ['Age', 'Gender', 'Country', 'Race', 'Family_Background', 'Radiation_History', 'Iodine_Deficiency', 'Smoke', 'Weight_Risk', 'Diabetes', 'Nodule_Size', 'TSH_Result', 'T4_Result', 'T3_Result', 'Age_tsh_interaction', 'Age_tsh_ratio', 'Nodule_Size_tsh_interaction', 'Family_Background_tsh_interaction', 'Iodine_Deficiency_tsh_interaction', 'Weight_Risk_tsh_interaction', 'Gender_tsh_interaction', 'Country_tsh_interaction']
training data did not have the following fields: Iodine_Deficiency_tsh_interaction, Country_tsh_interaction, Age_tsh_ratio, Family_Background_tsh_interaction, Weight_Risk_tsh_interaction, Nodule_Size_tsh_interaction, Age_tsh_interaction, Gender_tsh_interaction

Submission

In [9]:
submission = pd.read_csv('sample_submission.csv')
submission['Cancer'] = final_pred
submission.to_csv('baseline_submit.csv', index=False)